In [0]:
import random
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix,classification_report

In [0]:
w = 0.729844 
c1 = 1.496180 
c2 = 1.496180 

In [0]:
def model_score(x_train, y_train, x_test, y_test, classes = 'multi'):
    
    #x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
    model = DecisionTreeClassifier(random_state = 101)
    #print(type(model))
    model.fit(x_train,y_train)
    
    test_predictions = model.predict(x_test)
    if classes == 'multi':
        return f1_score(y_test, test_predictions, average = 'macro')
    else:
        return f1_score(y_test, test_predictions)

In [0]:
class Particle:
     
    def __init__(self,dimension):
        self.velocity = []
        self.pos = []
        self.pBest = []
        self.dimension = dimension
        n_chosen = random.randint(1,self.dimension-1)
        features = [0]*self.dimension
        for i in range(n_chosen):
            feat = random.randint(0,self.dimension-1)
            features[feat] = 1
        for i in range(self.dimension):
            self.pos.append(features[i])
            self.velocity.append(0.01*random.random())
            self.pBest.append(self.pos[i])
        #print(len(self.pos))
        return
 
    def updatePositions(self):
        for i in range(self.dimension):
            
            temp=1/(1+math.exp(-self.velocity[i]))
            if temp >= 0.5:
                self.pos[i]=1
            else:
                self.pos[i]=0
            
        #self.pos=1/(1+np.exp(-pos))
        return
 
    def updateVelocities(self, gBest):
        for i in range(self.dimension):
            r1 = random.random()
            r2 = random.random()
            social = c1 * r1 * (gBest[i] - self.pos[i])
            cognitive = c2 * r2 * (self.pBest[i] - self.pos[i])
            self.velocity[i] = (w * self.velocity[i]) + social + cognitive
            
        return 
   

In [0]:
class ParticleSwarmOptimizer:
     
    def __init__(self, X_train, y_train, X_test, y_test, swarmsize = 15):
        #self.solution = []
        self.swarm = []
        self.fitness_score=0
        self.f1_score=0
        self.alpha = 0.6
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.swarmSize = swarmsize
        for h in range(self.swarmSize):
            particle = Particle(dimension=X_train.shape[1])
            self.swarm.append(particle)
        return
 
    def optimize(self, iterations = 30, classes = 'multi'):
        for i in range(iterations):
            
            self.gBest = self.swarm[0].pos
        
            
            for j in range(self.swarmSize):
                pBest = self.swarm[j].pBest
                pBScore=self.f(pBest, classes = classes)
                gBScore=self.f(self.gBest, classes = classes)
                if pBScore[0] > gBScore[0]:
                    self.gBest = pBest
                    self.fitness_score = pBScore[0]
                    self.f1_score = pBScore[1]
            
            no_of_features = sum(self.gBest)
        
            for k in range(self.swarmSize):
                self.swarm[k].updateVelocities(self.gBest)
                self.swarm[k].updatePositions()

            for l in range(self.swarmSize):
                
                pBest = self.swarm[l].pBest
                pBScore = self.f(pBest, classes = classes)
                currfitness = pBScore[0]
                currf1 = pBScore[1]
                
                currPBest=self.f(self.swarm[l].pos, classes = classes)
                
                if currPBest[0] > currfitness:
                    self.swarm[l].pBest = self.swarm[l].pos
                    currfitness=currPBest[0]
                    currf1=currPBest[1]
               

        return no_of_features,self.f1_score, self.gBest
 
    def f(self,feature_array, classes):
    
        feature_index = [i for i in range(len(feature_array)) if feature_array[i] > 0]
        f1_score = model_score(self.X_train[:,feature_index], self.y_train, self.X_test[:,feature_index], self.y_test, classes = classes)
        fitness_score = self.alpha*f1_score + (1-self.alpha)*(X_train.shape[1]-len(feature_index))/X_train.shape[1]
        return [fitness_score,f1_score]
        

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
X_train=np.load("/content/drive/My Drive/Major2/X_train.npy")
y_train=np.load("/content/drive/My Drive/Major2/y_train.npy")
X_test=np.load("/content/drive/My Drive/Major2/X_test.npy")
y_test=np.load("/content/drive/My Drive/Major2/y_test.npy")

In [0]:
X_train = X_train.reshape(840, 208*176)
X_test = X_test.reshape(360, 208*176)

In [0]:
X_train.shape

(840, 36608)

In [0]:
pso = ParticleSwarmOptimizer(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

In [0]:
number_of_features, f1_score, feature_index = pso.optimize()

In [0]:
print(feature_index)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [0]:
len(feature_index)

36608

In [0]:
indices= [i for i,v in enumerate(feature_index) if v > 0]

In [0]:
len(indices)

1108

In [0]:
print(indices)

[9, 20, 87, 147, 150, 213, 230, 323, 364, 401, 433, 452, 464, 478, 483, 566, 592, 626, 692, 699, 750, 814, 831, 842, 866, 877, 892, 906, 910, 914, 930, 932, 941, 968, 982, 1093, 1116, 1227, 1280, 1301, 1309, 1369, 1406, 1443, 1450, 1458, 1497, 1527, 1539, 1578, 1645, 1663, 1719, 1768, 1774, 1804, 1896, 1927, 1929, 1940, 1946, 1953, 2005, 2018, 2060, 2086, 2198, 2209, 2225, 2227, 2243, 2272, 2281, 2294, 2303, 2369, 2380, 2431, 2485, 2487, 2489, 2525, 2543, 2592, 2648, 2686, 2736, 2796, 2818, 2833, 2846, 2871, 2900, 2908, 2927, 2950, 2967, 2973, 2982, 3003, 3178, 3223, 3274, 3346, 3361, 3427, 3440, 3449, 3493, 3506, 3545, 3547, 3561, 3589, 3614, 3640, 3657, 3664, 3699, 3705, 3752, 3795, 3797, 3835, 3838, 3890, 3983, 3990, 4044, 4046, 4053, 4054, 4073, 4089, 4095, 4175, 4192, 4198, 4244, 4310, 4322, 4363, 4376, 4441, 4567, 4639, 4684, 4779, 4830, 4895, 4933, 4960, 4985, 5039, 5103, 5113, 5210, 5214, 5249, 5253, 5267, 5307, 5330, 5344, 5404, 5442, 5505, 5506, 5533, 5553, 5563, 5621, 5652, 

In [0]:
X_train.shape

(840, 36608)

In [0]:
X_train_PSO=X_train[:,indices]
X_test_PSO=X_test[:,indices]

In [0]:
X_train_PSO.shape

(840, 1108)

In [0]:
np.save("X_train_PSO.npy",X_train_PSO)
np.save("y_train_PSO.npy",y_train)
np.save("X_test_PSO.npy",X_test_PSO)
np.save("y_test_PSO.npy",y_test)

In [0]:
from google.colab import files
files.download('X_train_PSO.npy')
files.download('X_test_PSO.npy')
files.download('y_train_PSO.npy')
files.download('y_test_PSO.npy')